# RedisTimeseriesLines Example: Market Data(OHLCV)

### Demonstrating a high-performance market price data downsampling mechanism using *redis* backend and `RedisTimeseriesLines`

In this example, we are going to maintain the data of some financial markets. We have chosen the `cryptocurrency` and `irx` for our example. Each market contain several instruments that we refer to them as symbols and we collect OHLCV(`open`, `high`, `low`, `close`, `volume`) data for each symbol.

The raw data is directly collected from the market with the resolution of seconds and we insert them in `raw` timeframe. Then we compress(downsample) the data to timeframes of `1m`, `1h` and `1d`. As the names `open`, `high`, `low`, `close`, `volume` implies, we use the `FIRST` aggregator for `open`, `MAX` for `high`, `MIN` for `low`, `LAST` for `close` and the `SUM` aggregator for `volume` to compress the data and build the appropriate timeframes of data.

We also want to keep `1m` data for just one week, `1h` for one month and respectively `1d` data for a year.
In this Example, we use the classifier 1(c1) to identify the market(here `cryptocurrency` or `irx`) and the classifier 2(c2) for the symbols.

In [1]:
import time, datetime, random
from pytz import timezone

from redis_timeseries_lines import RedisTimeseriesLines

settings = {
    'host': 'localhost',
    'port': 6379,
    'db': 13,
    'password': None,
}

class MarketData(RedisTimeseriesLines):
    _name = 'markets'
    _lines = ['open', 'high', 'low', 'close', 'volume']
    _timeframes = {
        'raw': {'retention_secs': 60*60*24*4}, # retention 4 days
        '1m': {'retention_secs': 60*60*24*7, 'bucket_size_secs': 60}, # retention 7 day; timeframe 60 secs
        '1h': {'retention_secs': 60*60*24*30, 'bucket_size_secs': 60*60}, # retention 1 month; timeframe 3600 secs
        '1d': {'retention_secs': 60*60*24*365, 'bucket_size_secs': 60*60*24}, # retention 1 year; timeframe 86400 secs
    }

    #compaction rules
    def _create_rule(self, c1:str, c2:str, line:str, timeframe_name:str, timeframe_specs:str, source_key:str, dest_key:str):
        if line == 'open':
            aggregation_type = 'first'
        elif line == 'close':
            aggregation_type = 'last'
        elif line == 'high':
            aggregation_type = 'max'
        elif line == 'low':
            aggregation_type = 'min'
        elif line == 'volume':
            aggregation_type = 'sum'
        else:
            return
        bucket_size_secs = timeframe_specs['bucket_size_secs']
        self._set_rule(source_key, dest_key, aggregation_type, bucket_size_secs)
    
    @staticmethod
    def print_data(data):
        for ts, open, high, low, close, volume in data:
            print(f"{datetime.datetime.fromtimestamp(ts, tz=timezone('UTC')):%Y-%m-%d %H:%M:%S}, open: {open}, high: {high}, low: {low}, close: {close}, volume: {volume}")

In [2]:
md = MarketData(**settings)

In this example, we don't create timeseries explicitly using the `create()` method. Instead, the series are created automatically while inserting data by turning on the `create_inplace` option.

In [3]:
crypto_btc = []
crypto_eth = []
irx_usd = []
# generating random data from 2020-01-01 to 2020-03-01; raw(seconds) resolution
for ts in range(1577836800, 1583020800, 60):
    btc = (random.randint(20000, 21000), random.randint(10000000, 20000000))
    eth = (random.randint(1500, 1600), random.randint(1000000, 2000000))
    usd = (random.randint(30000, 35000), random.randint(1000, 2000))
    crypto_btc.append([ts, btc[0], btc[0], btc[0], btc[0], btc[1]])
    crypto_eth.append([ts, eth[0], eth[0], eth[0], eth[0], eth[1]])
    irx_usd.append([ts, usd[0], usd[0], usd[0], usd[0], usd[1]])

# adding data
print(md.add(
    data=crypto_btc,
    c1='crypto',
    c2='btc',
    create_inplace=True,
)[1], 'records inserted for crypto:btc:raw')

print(md.add(
    data=crypto_eth,
    c1='crypto',
    c2='eth',
    create_inplace=True,
)[1], 'records inserted for crypto:eth:raw')

print(md.add(
    data=irx_usd,
    c1='irx',
    c2='usd',
    create_inplace=True,
)[1], 'records inserted for irx:usd:raw')


432000 records inserted for crypto:btc:raw
432000 records inserted for crypto:eth:raw
432000 records inserted for irx:usd:raw


Taking a look at `btc` data in `raw` timeframe. As you can see, data older than retention period are deleted:

In [4]:
data = md.read(
    c1='crypto',
    c2='btc',
    timeframe='raw',
)
md.print_data(data[1])

2020-02-25 23:59:00, open: 20572.0, high: 20572.0, low: 20572.0, close: 20572.0, volume: 11331537.0
2020-02-26 00:00:00, open: 20017.0, high: 20017.0, low: 20017.0, close: 20017.0, volume: 12882923.0
2020-02-26 00:01:00, open: 20923.0, high: 20923.0, low: 20923.0, close: 20923.0, volume: 14845320.0
2020-02-26 00:02:00, open: 20715.0, high: 20715.0, low: 20715.0, close: 20715.0, volume: 18209311.0
2020-02-26 00:03:00, open: 20130.0, high: 20130.0, low: 20130.0, close: 20130.0, volume: 18616553.0
2020-02-26 00:04:00, open: 20360.0, high: 20360.0, low: 20360.0, close: 20360.0, volume: 18129590.0
2020-02-26 00:05:00, open: 20621.0, high: 20621.0, low: 20621.0, close: 20621.0, volume: 13737272.0
2020-02-26 00:06:00, open: 20203.0, high: 20203.0, low: 20203.0, close: 20203.0, volume: 18722397.0
2020-02-26 00:07:00, open: 20497.0, high: 20497.0, low: 20497.0, close: 20497.0, volume: 12722421.0
2020-02-26 00:08:00, open: 20079.0, high: 20079.0, low: 20079.0, close: 20079.0, volume: 13530909.0


Making sure data is properly downsampled for the `1h` timeframe of `btc`. Just looking at last 10 records:

In [5]:
# sensor 1 last 10 data, timeframe 1h
data = md.read_last_n_records(
    c1='crypto',
    c2='btc',
    timeframe='1h',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-29 13:00:00, open: 20716.0, high: 20996.0, low: 20024.0, close: 20055.0, volume: 887865586.0
2020-02-29 14:00:00, open: 20662.0, high: 20955.0, low: 20005.0, close: 20264.0, volume: 909054096.0
2020-02-29 15:00:00, open: 20342.0, high: 20989.0, low: 20014.0, close: 20134.0, volume: 869658298.0
2020-02-29 16:00:00, open: 20575.0, high: 20993.0, low: 20000.0, close: 20198.0, volume: 918337539.0
2020-02-29 17:00:00, open: 20372.0, high: 20995.0, low: 20029.0, close: 20593.0, volume: 902469999.0
2020-02-29 18:00:00, open: 20317.0, high: 20998.0, low: 20020.0, close: 20267.0, volume: 897428031.0
2020-02-29 19:00:00, open: 20763.0, high: 20998.0, low: 20004.0, close: 20780.0, volume: 919447205.0
2020-02-29 20:00:00, open: 20241.0, high: 20968.0, low: 20008.0, close: 20390.0, volume: 859000785.0
2020-02-29 21:00:00, open: 20554.0, high: 20965.0, low: 20003.0, close: 20148.0, volume: 841459539.0
2020-02-29 22:00:00, open: 20028.0, high: 21000.0, low: 20015.0, close: 20330.0, volume: 93

And finally the `1d` timeframe for `btc`:

In [6]:
data = md.read_last_n_records(
    c1='crypto',
    c2='btc',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-19 00:00:00, open: 20996.0, high: 21000.0, low: 20001.0, close: 20899.0, volume: 21713494892.0
2020-02-20 00:00:00, open: 20407.0, high: 20999.0, low: 20001.0, close: 20572.0, volume: 21649732180.0
2020-02-21 00:00:00, open: 20655.0, high: 20999.0, low: 20000.0, close: 20210.0, volume: 21586585777.0
2020-02-22 00:00:00, open: 20839.0, high: 20999.0, low: 20000.0, close: 20470.0, volume: 21548996598.0
2020-02-23 00:00:00, open: 20431.0, high: 21000.0, low: 20000.0, close: 21000.0, volume: 21717670231.0
2020-02-24 00:00:00, open: 20843.0, high: 21000.0, low: 20000.0, close: 20375.0, volume: 21555332666.0
2020-02-25 00:00:00, open: 20947.0, high: 20999.0, low: 20000.0, close: 20572.0, volume: 21358709641.0
2020-02-26 00:00:00, open: 20017.0, high: 20999.0, low: 20000.0, close: 20620.0, volume: 21807551114.0
2020-02-27 00:00:00, open: 20317.0, high: 21000.0, low: 20000.0, close: 20171.0, volume: 21531302232.0
2020-02-28 00:00:00, open: 20206.0, high: 20999.0, low: 20000.0, close: 2

What about Ethereum?

In [7]:
data = md.read_last_n_records(
    c1='crypto',
    c2='eth',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-19 00:00:00, open: 1525.0, high: 1600.0, low: 1500.0, close: 1539.0, volume: 2170061446.0
2020-02-20 00:00:00, open: 1558.0, high: 1600.0, low: 1500.0, close: 1529.0, volume: 2162563677.0
2020-02-21 00:00:00, open: 1503.0, high: 1600.0, low: 1500.0, close: 1590.0, volume: 2166275502.0
2020-02-22 00:00:00, open: 1530.0, high: 1600.0, low: 1500.0, close: 1552.0, volume: 2169260556.0
2020-02-23 00:00:00, open: 1523.0, high: 1600.0, low: 1500.0, close: 1510.0, volume: 2171137650.0
2020-02-24 00:00:00, open: 1588.0, high: 1600.0, low: 1500.0, close: 1526.0, volume: 2164820801.0
2020-02-25 00:00:00, open: 1502.0, high: 1600.0, low: 1500.0, close: 1591.0, volume: 2173373542.0
2020-02-26 00:00:00, open: 1508.0, high: 1600.0, low: 1500.0, close: 1560.0, volume: 2167953279.0
2020-02-27 00:00:00, open: 1540.0, high: 1600.0, low: 1500.0, close: 1536.0, volume: 2169341657.0
2020-02-28 00:00:00, open: 1600.0, high: 1600.0, low: 1500.0, close: 1520.0, volume: 2146486135.0


We also had a market called `irx`, whats going on with it?

In [8]:
data = md.read_last_n_records(
    c1='irx',
    c2='usd',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-19 00:00:00, open: 32260.0, high: 34997.0, low: 30004.0, close: 31782.0, volume: 2148082.0
2020-02-20 00:00:00, open: 34475.0, high: 34999.0, low: 30000.0, close: 32224.0, volume: 2152698.0
2020-02-21 00:00:00, open: 31353.0, high: 34992.0, low: 30000.0, close: 30909.0, volume: 2184993.0
2020-02-22 00:00:00, open: 30458.0, high: 35000.0, low: 30001.0, close: 34704.0, volume: 2166722.0
2020-02-23 00:00:00, open: 31973.0, high: 34993.0, low: 30013.0, close: 31240.0, volume: 2162627.0
2020-02-24 00:00:00, open: 32847.0, high: 34992.0, low: 30003.0, close: 33333.0, volume: 2144194.0
2020-02-25 00:00:00, open: 31959.0, high: 34995.0, low: 30001.0, close: 30239.0, volume: 2165750.0
2020-02-26 00:00:00, open: 34780.0, high: 35000.0, low: 30002.0, close: 34021.0, volume: 2146596.0
2020-02-27 00:00:00, open: 33814.0, high: 35000.0, low: 30001.0, close: 33628.0, volume: 2156134.0
2020-02-28 00:00:00, open: 31246.0, high: 34999.0, low: 30004.0, close: 30958.0, volume: 2148934.0


## Other Commands
In the background, several keys are created in the redis db. To inspect the list of keys, run the following command:

In [9]:
md.query_index(return_key_names=True)[1]

['markets:crypto:btc:1d:close',
 'markets:crypto:btc:1d:high',
 'markets:crypto:btc:1d:low',
 'markets:crypto:btc:1d:open',
 'markets:crypto:btc:1d:volume',
 'markets:crypto:btc:1h:close',
 'markets:crypto:btc:1h:high',
 'markets:crypto:btc:1h:low',
 'markets:crypto:btc:1h:open',
 'markets:crypto:btc:1h:volume',
 'markets:crypto:btc:1m:close',
 'markets:crypto:btc:1m:high',
 'markets:crypto:btc:1m:low',
 'markets:crypto:btc:1m:open',
 'markets:crypto:btc:1m:volume',
 'markets:crypto:btc:raw:close',
 'markets:crypto:btc:raw:high',
 'markets:crypto:btc:raw:low',
 'markets:crypto:btc:raw:open',
 'markets:crypto:btc:raw:volume',
 'markets:crypto:eth:1d:close',
 'markets:crypto:eth:1d:high',
 'markets:crypto:eth:1d:low',
 'markets:crypto:eth:1d:open',
 'markets:crypto:eth:1d:volume',
 'markets:crypto:eth:1h:close',
 'markets:crypto:eth:1h:high',
 'markets:crypto:eth:1h:low',
 'markets:crypto:eth:1h:open',
 'markets:crypto:eth:1h:volume',
 'markets:crypto:eth:1m:close',
 'markets:crypto:eth:

You can inspect the info of each key by running the `stats` command as shown in the next cell.

> Note that each key have lablels properly filled with respective data(`c1`, `c2`, `...`), so you can take advantage of *redis multi-timeseries commands* like [TS.MRANGE](https://redis.io/commands/ts.mrange/)


In [10]:
md.stats('crypto', 'btc', 'raw', 'close').__dict__

{'rules': [[b'markets:crypto:btc:1m:close', 60000, b'LAST'],
  [b'markets:crypto:btc:1h:close', 3600000, b'LAST'],
  [b'markets:crypto:btc:1d:close', 86400000, b'LAST']],
 'source_key': None,
 'chunk_count': 4,
 'memory_usage': 17013,
 'total_samples': 5761,
 'labels': {'tl': 'markets',
  'c1': 'crypto',
  'c2': 'btc',
  'line': 'close',
  'timeframe': 'raw'},
 'retention_msecs': 345600000,
 'lastTimeStamp': 1583020740000,
 'first_time_stamp': 1582675140000,
 'chunk_size': 4096,
 'duplicate_policy': 'last'}